In [ ]:
pip install -U accelerate==0.29.3 peft==0.10.0 bitsandbytes==0.43.1 transformers==4.40.1 trl==0.8.6  datasets==2.19.0

In [1]:
import os
import torch
from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer
from langchain.llms import Ollama


/home/mghan/sopjt/git/stackoverflow_src/venv_stackoverflow_src/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

# QLoRA config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=False,
)


In [3]:
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



In [10]:
# Hugging Face Basic Model 한국어 모델
# base_model = "llama3.1:70b"
base_model = "/usr/share/d_ollama/.ollama/models"

# Custom Dataset ★ 본인이 hugging face 내 저장한 모델경로를 설정해야함 ★
hf_dataset = "myokyung/llama_test"

# new_model = "Llama-hkcode-Ko-3-8B"

In [11]:
dataset = load_dataset(hf_dataset, split="train")

# 데이터 확인
print( dataset[28] )

{'text': '<s>[INST] Here is the question about software development. The title of the question is "How can I add my credentials into .gitignore but still execute my python?". and body of the question is "<p>I am confused on how to add my credentials to .gitignore when they are in my config file. I have database credentials stored in my config file and they are used to execute my code below:</p>\n<pre><code>import os \nimport json\nimport platform\nimport logging\nimport pymysql as pm\nimport boto3\n\nclass ClassName:\n    env=None\n    config=None\n\n    def __init__(self, env_filename):\n        self.env=env_filename\n        self.config=self.get_config()\n\n    def get_config(self):\n        with open(self.env) as file_in:\n            return json.load(file_in)\n        \n    def is_Windows():\n        if &quot;win&quot; in (platform.system().lower()):\n            return True\n        else:\n            return False\n\n    def DB_connection(self):\n        logger = logging.getLogger

In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import SFTTrainer

model = AutoModelForCausalLM.from_pretrained(base_model)
tokenizer = AutoTokenizer.from_pretrained(base_model)

trainer = SFTTrainer(
    model=base_model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args=TrainingArguments(
        per_device_train_batch_size=4,
        num_train_epochs=3,
        learning_rate=5e-5,
    ),
)

trainer.train()

OSError: /usr/share/d_ollama/.ollama/models does not appear to have a file named config.json. Checkout 'https://huggingface.co//usr/share/d_ollama/.ollama/models/tree/None' for available files.

In [ ]:
from langchain.llms import Ollama

llm = Ollama(base_url="http://localhost:11434", model="llama3.1:70b")
